# ETLs

In [3]:
import pandas as pd
import sqlite3

sales_con = sqlite3.connect("go_sales.sqlite")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", sales_con)

try:
    country          = pd.read_sql_query("SELECT * FROM country;", sales_con)
    order_details    = pd.read_sql_query("SELECT * FROM order_details;", sales_con)
    order_header     = pd.read_sql_query("SELECT * FROM order_header;", sales_con)
    order_method     = pd.read_sql_query("SELECT * FROM order_method;", sales_con)
    product          = pd.read_sql_query("SELECT * FROM product;", sales_con)
    product_line     = pd.read_sql_query("SELECT * FROM product_line;", sales_con)
    product_type     = pd.read_sql_query("SELECT * FROM product_type;", sales_con)
    retailer_site    = pd.read_sql_query("SELECT * FROM retailer_site;", sales_con)
    return_reason    = pd.read_sql_query("SELECT * FROM return_reason;", sales_con)
    returned_item    = pd.read_sql_query("SELECT * FROM returned_item;", sales_con)
    sales_branch     = pd.read_sql_query("SELECT * FROM sales_branch;", sales_con)
    sales_staff      = pd.read_sql_query("SELECT * FROM sales_staff;", sales_con)
    SALES_TARGETData = pd.read_sql_query("SELECT * FROM SALES_TARGETData;", sales_con)
    sqlite_sequence  = pd.read_sql_query("SELECT * FROM sqlite_sequence;", sales_con)

    print("Imported data succesfully!")
except:
    print("That didn't work :(")
    
tables

Imported data succesfully!


,name
0,country
1,order_details
2,order_header
3,order_method
4,product
5,product_line
6,product_type
7,retailer_site
8,return_reason
9,returned_item


In [13]:
product_etl = pd.merge(product, product_type, on="PRODUCT_TYPE_CODE")\
    [[
        "PRODUCT_NAME", "PRODUCT_IMAGE", "PRODUCT_NUMBER",
        "DESCRIPTION", "LANGUAGE", "PRODUCTION_COST",
        "MARGIN", "INTRODUCTION_DATE", "PRODUCT_LINE_CODE"
    ]]

product_etl = pd.merge(product_etl, product_line, on="PRODUCT_LINE_CODE")\
    .rename(columns={
        "PRODUCT_NAME": "PRODUCT_name",
        "PRODUCT_IMAGE": "PRODUCT_image",
        "PRODUCT_NUMBER": "PRODUCT_id",
        "DESCRIPTION": "PRODUCT_description",
        "LANGUAGE": "PRODUCT_LANGUAGE_name",
        "PRODUCTION_COST": "PRODUCT_PRODUCTION_COST_cost",
        "MARGIN": "PRODUCT_MARGIN_percentage",
        "INTRODUCTION_DATE": "PRODUCT_INTRODUCTION_DATE_date",
        "PRODUCT_LINE_CODE": "PRODUCT_LINE_code",
        "PRODUCT_LINE_EN": "PRODUCT_LINE_name"
    })

product_etl

,PRODUCT_name,PRODUCT_image,PRODUCT_id,PRODUCT_description,PRODUCT_LANGUAGE_name,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_percentage,PRODUCT_INTRODUCTION_DATE_date,PRODUCT_LINE_code,PRODUCT_LINE_name,TRIAL888
0,TrailChef Water Bag,P01CE1CG1.jpg,1,"Lightweight, collapsible bag to carry liquids ...",EN,4,.33,15-2-2011,1,Camping Equipment,T
1,TrailChef Utensils,P10CE1CG1.jpg,10,"Spoon, fork and knife set made of a light yet ...",EN,10,.4,15-2-2011,1,Camping Equipment,T
2,Insect Bite Relief,P100OP4FA17.jpg,100,The Insect Bite Relief helps the itching and s...,EN,3,.5,15-2-2011,4,Outdoor Protection,T
3,Hailstorm Steel Irons,P101GE5IR18.jpg,101,Iron is 17-4 stainless steel. Shafts are grap...,EN,305.54,.43,15-12-2019,5,Golf Equipment,T
4,Hailstorm Titanium Irons,P102GE5IR18.jpg,102,Made entirely of pure titanium. The ultimate i...,EN,380.95,.51,10-12-2019,5,Golf Equipment,T
...,...,...,...,...,...,...,...,...,...,...,...
110,Sun Shield,P91OP4SS16.jpg,95,"PABA free sunscreen, SPF 30, poison oak and iv...",EN,3,.5,15-2-2011,4,Outdoor Protection,T
111,Compact Relief Kit,P96OP4FA17.jpg,96,A personal first aid kit is recommended for ev...,EN,16.43,.28,15-2-2011,4,Outdoor Protection,T
112,Deluxe Family Relief Kit,P96OP4FA17.jpg,97,A complete medical kit suitable for families w...,EN,25,.28,5-3-2013,4,Outdoor Protection,T
113,Calamine Relief,P98OP4FA17.jpg,98,Use the Calamine Relief for allergic skin reac...,EN,3,.5,15-2-2011,4,Outdoor Protection,T
